In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
# from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import lightgbm as lgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score


In [3]:
db_prm= pd.read_csv("Data/Primary Data.csv")
db_mtch=pd.read_csv("Data/Match Scorecard.csv")
db_bat=pd.read_csv('Data/Batsmen Data.csv')
db_bowl=pd.read_csv('Data/Bowler Data.csv')
db_prmTest= pd.read_csv('Data/PrmTest.csv')

In [4]:
db_prm['match_dt'] = pd.to_datetime(db_prm['match_dt'])

db_mtch['match_dt'] = pd.to_datetime(db_mtch['match_dt'],format='mixed')
db_bat['match_dt'] = pd.to_datetime(db_bat['match_dt'],format='%d-%m-%Y')
db_bowl['match_dt'] = pd.to_datetime(db_bowl['match_dt'])

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_836\2849964513.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  db_prm['match_dt'] = pd.to_datetime(db_prm['match_dt'])


In [5]:
db_prm["winner_num"]=db_prm.apply(lambda row: 1 if row['winner']==row['team1'] else 0, axis=1)


Lets begin with features

In [6]:
## Toss advantage feature

def calculate_toss_advantage(db_mtch, db_prm):
    # Initialize a dictionary to store toss win and match win counts
    ground_toss_win_stats = {}

    # Calculate win percentages for toss-winning teams on each ground up to the current match date
    for i, row in db_mtch.iterrows():
        match_date = row['match_dt']
        ground_id = row['ground_id']
        toss_winner_name = row['toss winner']
        winner_name = row['winner']

        if ground_id not in ground_toss_win_stats:
            ground_toss_win_stats[ground_id] = {'toss_wins': 0, 'toss_and_match_wins': 0}

        # Increment toss win count
        ground_toss_win_stats[ground_id]['toss_wins'] += 1

        # Increment count if the toss winner also won the match
        if toss_winner_name == winner_name:
            ground_toss_win_stats[ground_id]['toss_and_match_wins'] += 1

    # Function to calculate toss win percentage
    def get_toss_win_percentage(ground_stats):
        if ground_stats['toss_wins'] == 0:
            return 0
        return ground_stats['toss_and_match_wins'] / ground_stats['toss_wins']

    # Create new column in db_prm for toss win percentages
    toss_win_percentages = []

    for i, row in db_prm.iterrows():
        match_date = row['match_dt']
        ground_id = row['ground_id']

        # Calculate toss win percentage up to the current match date
        if ground_id in ground_toss_win_stats:
            ground_stats = ground_toss_win_stats[ground_id]
            toss_win_percentage = get_toss_win_percentage(ground_stats)
        else:
            toss_win_percentage = 0

        toss_win_percentages.append(toss_win_percentage)

    # Add new column to the primary dataset
    db_prm['toss_win_percentage'] = toss_win_percentages

    # Calculate toss advantage for team1 and team2
    db_prm['toss_adv_team1'] = db_prm.apply(lambda row: row['toss_win_percentage'] if row['toss winner'] == row['team1'] else 0, axis=1)
    db_prm['toss_adv_team2'] = db_prm.apply(lambda row: row['toss_win_percentage'] if row['toss winner'] == row['team2'] else 0, axis=1)
    db_prm['toss_adv_diff']=db_prm['toss_adv_team1']-db_prm['toss_adv_team2']

    # Return the updated db_prm dataframe
    return db_prm

db_prm = calculate_toss_advantage(db_mtch, db_prm)

In [7]:
## RUN/WICKET RATIO
def calculate_team_stats(df, team_id_col, runs_col, wickets_col, match_date_col):
    team_stats = {}
    for i, row in df.iterrows():
        team_id = row[team_id_col]
        match_date = row[match_date_col]
        if team_id not in team_stats:
            team_stats[team_id] = {'runs': [], 'wickets': [], 'margin': []}
        
        previous_matches = df[(df[team_id_col] == team_id) & (df[match_date_col] < match_date)].tail(5)
        team_stats[team_id]['runs'].append(previous_matches[runs_col].mean())
        team_stats[team_id]['wickets'].append(previous_matches[wickets_col].mean())
        
    
    return team_stats

# Calculate stats for team1 and team2
team1_stats = calculate_team_stats(db_mtch, 'team1_id', 'inning1_runs', 'inning1_wickets',  'match_dt')
team2_stats = calculate_team_stats(db_mtch, 'team2_id', 'inning2_runs', 'inning2_wickets',  'match_dt')

# Function to add ratio features and average runs to db_prm
def add_ratio_features(db_prm, team1_stats, team2_stats):
    avg_runs_team1_last_5 = []
    avg_runs_team2_last_5 = []
    run_avg_diff=[]
    wicket_avg_diff=[]
    avg_runRatio=[]
    avg_wicketRatio=[]
    
    for i, row in db_prm.iterrows():
        match_date = row['match_dt']
        team1_id = row['team1_id']
        team2_id = row['team2_id']
        
        # Filter previous matches
        previous_team1_matches = db_mtch[(db_mtch['team1_id'] == team1_id) & (db_mtch['match_dt'] < match_date)].tail(5)
        previous_team2_matches = db_mtch[(db_mtch['team2_id'] == team2_id) & (db_mtch['match_dt'] < match_date)].tail(5)
        
        # Calculate averages
        team1_avg_runs = previous_team1_matches['inning1_runs'].mean() if not previous_team1_matches.empty else 0
        team2_avg_runs = previous_team2_matches['inning2_runs'].mean() if not previous_team2_matches.empty else 0
        team1_avg_wickets = previous_team1_matches['inning1_wickets'].mean() if not previous_team1_matches.empty else 0
        team2_avg_wickets = previous_team2_matches['inning2_wickets'].mean() if not previous_team2_matches.empty else 0
       


        run_ratio=0.5
        wicket_ratio=0.5
        # Add average runs & adv_diff to list
        avg_runs_team1_last_5.append(team1_avg_runs)
        avg_runs_team2_last_5.append(team2_avg_runs)
        run_avg_diff.append(team1_avg_runs-team2_avg_runs)
        wicket_diff=0
        if team1_avg_wickets!=0 and team2_avg_wickets!=0:
            wicket_diff= team1_avg_wickets-team2_avg_wickets
        wicket_avg_diff.append(wicket_diff)

        if team2_avg_runs!=0:
            run_ratio=team1_avg_runs/ team2_avg_runs
        
        if team2_avg_wickets!=0:
            wicket_ratio=team1_avg_wickets/team2_avg_wickets
        
        avg_runRatio.append(run_ratio)
        avg_wicketRatio.append(wicket_ratio)    
    
    db_prm['avg_runs_team1_last_5'] = avg_runs_team1_last_5
    db_prm['avg_runs_team2_last_5'] = avg_runs_team2_last_5
    db_prm['run_avg_diff']= run_avg_diff
    db_prm['wicket_avg_diff']=wicket_avg_diff
    db_prm['Run ratio']=avg_runRatio
    db_prm['Wicket_ratio']=avg_wicketRatio
    
    


add_ratio_features(db_prm, team1_stats, team2_stats)       #function call

In [8]:
## Ground advantage to both teams

db_prm['ground_avg_runs_last15'].fillna(db_prm['ground_avg_runs_last15'].median(), inplace=True)
# Function to add 'batting_to_ground_avg_run_ratio' to db_prm
def add_batting_to_ground_avg_run_ratio(db_prm):
    ground_adv_team1 = []
    ground_adv_team2 = []
    
    for i, row in db_prm.iterrows():
        team1_avg_last5 = row['avg_runs_team1_last_5']
        team2_avg_last5 = row['avg_runs_team2_last_5']
        ground_avg_runs = row['ground_avg_runs_last15']
        team1 = row['team1']
        team2=row['team2']
        toss_winner=row['toss winner']
        
        ratio1 = team1_avg_last5 / ground_avg_runs if team1==toss_winner else 0
        ground_adv_team1.append(ratio1)
        
        ratio2 = team2_avg_last5 / ground_avg_runs if team2==toss_winner else 0
        ground_adv_team2.append(ratio2)
  
    db_prm['ground_adv_team1'] = ground_adv_team1
    db_prm['ground_adv_team2'] = ground_adv_team2
    db_prm['ground_adv_diff']=db_prm['ground_adv_team1']-db_prm['ground_adv_team2']

# Add the new ratio feature to primary dataset
add_batting_to_ground_avg_run_ratio(db_prm)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_836\1709139784.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  db_prm['ground_avg_runs_last15'].fillna(db_prm['ground_avg_runs_last15'].median(), inplace=True)


In [9]:
def dataframe(db_mtch, team1_id,team2_id, match_dt):
        # Ensure 'match_dt' column is in datetime format
        if db_mtch['match_dt'].dtype == object:
            db_mtch['match_dt'] = pd.to_datetime(db_mtch['match_dt'], format='%d-%m-%Y', errors='coerce')

        # Ensure 'match_dt' input is in datetime format
        match_dt = pd.to_datetime(match_dt, format='%d-%m-%Y', errors='coerce')


        # Get matches for team1 and team2
        team1_matches = db_mtch[db_mtch['team1_id'] == team1_id]
        team2_matches = db_mtch[db_mtch['team2_id'] == team2_id]

        # Sort matches by date
        team1_matches_sorted = team1_matches.sort_values(by='match_dt', ascending=True)
        team2_matches_sorted = team2_matches.sort_values(by='match_dt', ascending=True)

        # Get the last three matches for each team before the given match date
        team1_last_three_matches = team1_matches_sorted[team1_matches_sorted['match_dt'] < match_dt].tail(3)['match id'].tolist()
        team2_last_three_matches = team2_matches_sorted[team2_matches_sorted['match_dt'] < match_dt].tail(3)['match id'].tolist()

        # Ensure there are exactly three elements in each list by adding None if necessary
        if len(team1_last_three_matches) < 3:
            team1_last_three_matches = [None] * (3 - len(team1_last_three_matches)) + team1_last_three_matches
        if len(team2_last_three_matches) < 3:
            team2_last_three_matches = [None] * (3 - len(team2_last_three_matches)) + team2_last_three_matches

        # Get the rosters for the last three matches
        team1_last_three_rosters = []
        for m_id in team1_last_three_matches:
            if m_id is not None:
                match_row = db_mtch[db_mtch['match id'] == m_id].iloc[0]
                team1_last_three_rosters.append(match_row['team1_roster_ids'] if pd.notna(match_row['team1_roster_ids']) else None)
            else:
                team1_last_three_rosters.append(None)

        team2_last_three_rosters = []
        for m_id in team2_last_three_matches:
            if m_id is not None:
                match_row = db_mtch[db_mtch['match id'] == m_id].iloc[0]
                team2_last_three_rosters.append(match_row['team2_roster_ids'] if pd.notna(match_row['team2_roster_ids']) else None)
            else:
                team2_last_three_rosters.append(None)

        # Create DataFrames for the last three matches and rosters
        df1 = pd.DataFrame([team1_last_three_matches, team1_last_three_rosters], index=['Match_ID', 'Roster_ID'],
                           columns=['Last3rd_Match', 'Last2nd_Match', 'Last_Match'])
        df1.index.name = 'Feature'

        df2 = pd.DataFrame([team2_last_three_matches, team2_last_three_rosters], index=['Match_ID', 'Roster_ID'],
                           columns=['Last3rd_Match', 'Last2nd_Match', 'Last_Match'])
        df2.index.name = 'Feature'

        return df1, df2

In [10]:
def calculate_nrr_from_matches(match_ids, team_id, db_mtch):
    total_runs_scored = 0
    total_overs_faced = 0
    total_runs_conceded = 0
    total_overs_bowled = 0
    total_wickets_taken = 0
    total_wickets_lost = 0

    for match_id in match_ids:
        if match_id is not None:
            match_rows = db_mtch[db_mtch['match id'] == match_id]
            if not match_rows.empty:
                match_row = match_rows.iloc[0]
                if match_row['team1_id'] == team_id:
                    # Team batted first
                    total_runs_scored += match_row['inning1_runs']
                    total_overs_faced += match_row['inning1_balls'] / 6  # Convert balls to overs
                    total_runs_conceded += match_row['inning2_runs']
                    total_overs_bowled += match_row['inning2_balls'] / 6  # Convert balls to overs
                    total_wickets_taken += match_row['inning2_wickets']
                    total_wickets_lost += match_row['inning1_wickets']
                else:
                    # Team batted second
                    total_runs_scored += match_row['inning2_runs']
                    total_overs_faced += match_row['inning2_balls'] / 6  # Convert balls to overs
                    total_runs_conceded += match_row['inning1_runs']
                    total_overs_bowled += match_row['inning1_balls'] / 6  # Convert balls to overs
                    total_wickets_taken += match_row['inning1_wickets']
                    total_wickets_lost += match_row['inning2_wickets']

    if total_overs_faced == 0 or total_overs_bowled == 0:
        return 0, 0, 0  # Return zeros if no matches contribute to NRR calculation

    nrr = (total_runs_scored / total_overs_faced) - (total_runs_conceded / total_overs_bowled)
    wicket_runs = (total_wickets_taken / total_runs_conceded) - (total_wickets_lost / total_runs_scored)
    wicket_overs = (total_wickets_taken / total_overs_bowled) - (total_wickets_lost / total_overs_faced)
    
    return nrr, wicket_runs, wicket_overs  # Return all three values as a tuple


def add_nrr_features(db_prm, db_mtch):
    team1_nrr_list = []
    team2_nrr_list = []
    team1_wicket_runs_list=[]
    team2_wicket_runs_list=[]
    team1_wicket_overs_list=[]
    team2_wicket_overs_list=[]

    for i, row in db_prm.iterrows():
        team1_id = row['team1_id']
        team2_id = row['team2_id']
        match_dt = row['match_dt']

        df1, df2 = dataframe(db_mtch, team1_id, team2_id, match_dt)

        # Extract the last three match IDs from df1 and df2
        team1_last_three_matches = df1.loc['Match_ID'].tolist()
        team2_last_three_matches = df2.loc['Match_ID'].tolist()

        # Calculate NRR based on the extracted match IDs
        team1_nrr,team1_wicket_runs,team1_wicket_overs = calculate_nrr_from_matches(team1_last_three_matches, team1_id, db_mtch)
        team2_nrr,team2_wicket_runs,team2_wicket_overs = calculate_nrr_from_matches(team2_last_three_matches, team2_id, db_mtch)

        team1_nrr_list.append(team1_nrr)
        team2_nrr_list.append(team2_nrr)
        team1_wicket_runs_list.append(team1_wicket_runs)
        team2_wicket_runs_list.append(team2_wicket_runs)
        team1_wicket_overs_list.append(team1_wicket_overs)
        team2_wicket_overs_list.append(team2_wicket_overs)
        
        

    db_prm['team1_nrr'] = team1_nrr_list
    db_prm['team2_nrr'] = team2_nrr_list
    db_prm['team1_wicket_runs']=team1_wicket_runs_list
    db_prm['team2_wicket_runs']=team2_wicket_runs_list
    db_prm['team1_wicket_overs']=team1_wicket_overs_list
    db_prm['team2_wicket_overs']=team2_wicket_overs_list
    db_prm['nrr_diff']= db_prm['team1_nrr'] - db_prm['team2_nrr']
    db_prm['wicket_runs_diff']=db_prm['team1_wicket_runs'] -db_prm['team2_wicket_runs']
    db_prm['wicket_overs_diff']= db_prm['team1_wicket_overs'] - db_prm['team2_wicket_overs']

# Assuming db_prm and db_mtch are your DataFrames
add_nrr_features(db_prm, db_mtch)

In [11]:
## FUNCTION TO CALCULATE PPW RATIOs
def PPW_FinishSR_adder(db_prm, db_mtch, db_bat):
    
    def calculate_power_play_wickets(match_id, roster_id, db_bat):
        if pd.isna(match_id) or roster_id is None:
            return 0

        roster_ids = roster_id.split(':')
        count = 0
        for player_id in roster_ids:
            player_id = float(player_id)  # Convert to float to match the db_bat data
            matching_rows = db_bat[(db_bat['batsman_id'] == player_id) & (db_bat['match id'] == match_id)]
            count += (matching_rows['over_faced_first'] < 7).sum()
        power_play_wickets = max(count - 2, 0)
        return power_play_wickets


    def calculate_avg_finishing_sr(match_id, roster_id, db_bat):
        if pd.isna(match_id) or roster_id is None:
            return 0

        roster_ids = roster_id.split(':')
        strike_rates = []
        for player_id in roster_ids:
            player_id = float(player_id)  # Convert to float to match the db_bat data
            matching_rows = db_bat[(db_bat['batsman_id'] == player_id) & (db_bat['match id'] == match_id)]
            if not matching_rows.empty:
                over_faced_first_values = matching_rows['over_faced_first'].tolist()
                # Sort and take the top three values of over_faced_first
                top_three_values=[]
                if len(over_faced_first_values)>=3:
                    top_three_values = sorted(over_faced_first_values, reverse=True)[:3]
                # Sum up the strike rates for these top three values

                if len(over_faced_first_values)<3:
                    top_three_values = sorted(over_faced_first_values, reverse=True)[:2]
                
                sum_strike_rates = 0
                for value in top_three_values:
                    sum_strike_rates += matching_rows[matching_rows['over_faced_first'] == value]['strike_rate'].iloc[0]
                strike_rates.append(sum_strike_rates)

        if strike_rates:
            avg_finishing_sr = sum(strike_rates) / len(strike_rates)
        else:
            avg_finishing_sr = 0

        return avg_finishing_sr

    def add_avg_power_play_wickets(db_prm, db_mtch, db_bat):
        avg_ppw_team1 = []
        avg_ppw_team2 = []
        avg_finishing_sr_team1=[]
        avg_finishing_sr_team2=[]

        for index, row in db_prm.iterrows():
            team1_id = row['team1_id']
            team2_id=row['team2_id']
            match_dt = row['match_dt']

            df_team1, df_team2 = dataframe(db_mtch, team1_id,team2_id, match_dt)

            # Extract match and roster IDs for team1
            roster_id_last3rd_team1 = df_team1.loc['Roster_ID', 'Last3rd_Match']
            roster_id_last2nd_team1 = df_team1.loc['Roster_ID', 'Last2nd_Match']
            roster_id_last_team1 = df_team1.loc['Roster_ID', 'Last_Match']

            match_id_last3rd_team1 = df_team1.loc['Match_ID', 'Last3rd_Match']
            match_id_last2nd_team1 = df_team1.loc['Match_ID', 'Last2nd_Match']
            match_id_last_team1 = df_team1.loc['Match_ID', 'Last_Match']

            # Calculate power play wickets for team1
            ppw_last3rd_team1 = calculate_power_play_wickets(match_id_last3rd_team1, roster_id_last3rd_team1, db_bat)
            ppw_last2nd_team1 = calculate_power_play_wickets(match_id_last2nd_team1, roster_id_last2nd_team1, db_bat)
            ppw_last_team1 = calculate_power_play_wickets(match_id_last_team1, roster_id_last_team1, db_bat)
            avg_team1=(ppw_last3rd_team1 + ppw_last2nd_team1 + ppw_last_team1) / 3


            # Calculate average finishing strike rate for team1
            avg_finishing_sr_last3rd_team1 = calculate_avg_finishing_sr(match_id_last3rd_team1, roster_id_last3rd_team1, db_bat)
            avg_finishing_sr_last2nd_team1 = calculate_avg_finishing_sr(match_id_last2nd_team1, roster_id_last2nd_team1, db_bat)
            avg_finishing_sr_last_team1 = calculate_avg_finishing_sr(match_id_last_team1, roster_id_last_team1, db_bat)
            avg_finishing_sr_team1.append((avg_finishing_sr_last3rd_team1 + avg_finishing_sr_last2nd_team1 + avg_finishing_sr_last_team1) / 3)

            # Extract match and roster IDs for team2
            roster_id_last3rd_team2 = df_team2.loc['Roster_ID', 'Last3rd_Match']
            roster_id_last2nd_team2 = df_team2.loc['Roster_ID', 'Last2nd_Match']
            roster_id_last_team2 = df_team2.loc['Roster_ID', 'Last_Match']

            match_id_last3rd_team2 = df_team2.loc['Match_ID', 'Last3rd_Match']
            match_id_last2nd_team2 = df_team2.loc['Match_ID', 'Last2nd_Match']
            match_id_last_team2 = df_team2.loc['Match_ID', 'Last_Match']

            # Calculate power play wickets for team2
            ppw_last3rd_team2 = calculate_power_play_wickets(match_id_last3rd_team2, roster_id_last3rd_team2, db_bat)
            ppw_last2nd_team2 = calculate_power_play_wickets(match_id_last2nd_team2, roster_id_last2nd_team2, db_bat)
            ppw_last_team2 = calculate_power_play_wickets(match_id_last_team2, roster_id_last_team2, db_bat)
            avg_team2=(ppw_last3rd_team2 + ppw_last2nd_team2 + ppw_last_team2) / 3

            # Calculate average finishing strike rate for team2
            avg_finishing_sr_last3rd_team2 = calculate_avg_finishing_sr(match_id_last3rd_team2, roster_id_last3rd_team2, db_bat)
            avg_finishing_sr_last2nd_team2 = calculate_avg_finishing_sr(match_id_last2nd_team2, roster_id_last2nd_team2, db_bat)
            avg_finishing_sr_last_team2 = calculate_avg_finishing_sr(match_id_last_team2, roster_id_last_team2, db_bat)
            avg_finishing_sr_team2.append((avg_finishing_sr_last3rd_team2 + avg_finishing_sr_last2nd_team2 + avg_finishing_sr_last_team2) / 3)

 
            avg_ppw_team1.append(avg_team1)
            avg_ppw_team2.append(avg_team2)
            
        db_prm['PPW_Avg_team1'] = avg_ppw_team1
        db_prm['PPW_Avg_team2'] = avg_ppw_team2
        db_prm['PPW_Avg_Diff']=db_prm['PPW_Avg_team1']-db_prm['PPW_Avg_team2']

        db_prm['Avg_Finishing_SR_team1'] = avg_finishing_sr_team1
        db_prm['Avg_Finishing_SR_team2'] = avg_finishing_sr_team2
        db_prm['Avg_Finishing_SR_Diff'] = db_prm['Avg_Finishing_SR_team1'] - db_prm['Avg_Finishing_SR_team2']


        return db_prm

    db_prm = add_avg_power_play_wickets(db_prm, db_mtch, db_bat)
    return db_prm

In [12]:
db_prm = PPW_FinishSR_adder(db_prm,db_mtch,db_bat)

In [13]:
def BoundaryRunsFraction_Adder(db_prm, db_mtch, db_bat):

    def calculate_boundary_fraction(match_id, roster_id, db_bat):
        if pd.isna(match_id) or pd.isna(roster_id):
            return 0
        roster_ids = roster_id.split(':')
        total_runs = 0
        boundary_runs = 0
        for player_id in roster_ids:
            player_id = float(player_id)
            matching_rows = db_bat[(db_bat['batsman_id'] == player_id) & (db_bat['match id'] == match_id)]
            if not matching_rows.empty:
                total_runs += matching_rows['runs'].sum()
                boundary_runs += (matching_rows['Fours'] * 4).sum() + (matching_rows['Sixes'] * 6).sum()
        if total_runs == 0:
            return 0
        return boundary_runs / total_runs

    def add_boundary_fraction(db_prm, db_mtch, db_bat):
        avg_team1 = []
        avg_team2 = []

        for index, row in db_prm.iterrows():
            match_id = row['match id']
            match_dt = row['match_dt']
            team1_id = row['team1_id']
            team2_id = row['team2_id']

            df_team1, df_team2 = dataframe(db_mtch, team1_id, team2_id, match_dt)

            # Extract match and roster IDs for team1
            roster_id_last3rd_team1 = df_team1.loc['Roster_ID', 'Last3rd_Match']
            roster_id_last2nd_team1 = df_team1.loc['Roster_ID', 'Last2nd_Match']
            roster_id_last_team1 = df_team1.loc['Roster_ID', 'Last_Match']

            match_id_last3rd_team1 = df_team1.loc['Match_ID', 'Last3rd_Match']
            match_id_last2nd_team1 = df_team1.loc['Match_ID', 'Last2nd_Match']
            match_id_last_team1 = df_team1.loc['Match_ID', 'Last_Match']

            # Calculate boundary runs fraction for team1
            fraction_last3rd_team1 = calculate_boundary_fraction(match_id_last3rd_team1, roster_id_last3rd_team1, db_bat)
            fraction_last2nd_team1 = calculate_boundary_fraction(match_id_last2nd_team1, roster_id_last2nd_team1, db_bat)
            fraction_last_team1 = calculate_boundary_fraction(match_id_last_team1, roster_id_last_team1, db_bat)
            avg_fraction1 = (fraction_last3rd_team1 + fraction_last2nd_team1 + fraction_last_team1) / 3

            # Extract match and roster IDs for team2
            roster_id_last3rd_team2 = df_team2.loc['Roster_ID', 'Last3rd_Match']
            roster_id_last2nd_team2 = df_team2.loc['Roster_ID', 'Last2nd_Match']
            roster_id_last_team2 = df_team2.loc['Roster_ID', 'Last_Match']

            match_id_last3rd_team2 = df_team2.loc['Match_ID', 'Last3rd_Match']
            match_id_last2nd_team2 = df_team2.loc['Match_ID', 'Last2nd_Match']
            match_id_last_team2 = df_team2.loc['Match_ID', 'Last_Match']

            # Calculate boundary runs fraction for team2
            fraction_last3rd_team2 = calculate_boundary_fraction(match_id_last3rd_team2, roster_id_last3rd_team2, db_bat)
            fraction_last2nd_team2 = calculate_boundary_fraction(match_id_last2nd_team2, roster_id_last2nd_team2, db_bat)
            fraction_last_team2 = calculate_boundary_fraction(match_id_last_team2, roster_id_last_team2, db_bat)
            avg_fraction2 = (fraction_last3rd_team2 + fraction_last2nd_team2 + fraction_last_team2) / 3

            avg_team1.append(  avg_fraction1)
            avg_team2.append(avg_fraction2)

        db_prm['BoundaryFraction_team1'] = avg_team1
        db_prm['BoundaryFraction_team2'] = avg_team2
        db_prm['BoundaryFraction_Diff'] = db_prm['BoundaryFraction_team1'] - db_prm['BoundaryFraction_team2']

        return db_prm

    db_prm = add_boundary_fraction(db_prm, db_mtch, db_bat)
    return db_prm

In [14]:
db_prm=BoundaryRunsFraction_Adder(db_prm, db_mtch, db_bat)

In [15]:
import pandas as pd

def SR1_Adder(db_prm, db_mtch, db_bat):
    
    def calculate_avg_SR_and_50plus_count(player_id, match_dt, db_bat):
        # Get all matches for the player before the current match date
        player_matches = db_bat[(db_bat['batsman_id'] == player_id) & (db_bat['match_dt'] < match_dt)]
        
        # Sort matches by date and get the last 5 matches
        player_matches_sorted = player_matches.sort_values(by='match_dt', ascending=False).head(5)
        
        # Calculate the average strike rate and count 50+ scores
        if not player_matches_sorted.empty:
            avg_SR = player_matches_sorted['strike_rate'].mean()
            fifty_plus_count = (player_matches_sorted['runs'] > 30).sum()
            return avg_SR, fifty_plus_count
        else:
            return 0, 0

    def Add_StrikeRate(db_prm, db_mtch, db_bat):
        Avg_SR_team1 = []
        Avg_SR_team2 = []
        SR_Adv1 = []
        SR_Adv2 = []
        Fifty_plus_count_team1 = []
        Fifty_plus_count_team2 = []

        for index, row in db_prm.iterrows():
            match_id = row['match id']
            match_dt = row['match_dt']
            team1_id = row['team1_id']
            team2_id = row['team2_id']
            roster_id_team1 = row['team1_roster_ids']
            roster_id_team2 = row['team2_roster_ids']

            # Split and store the player IDs from the roster
            roster_ids_team1 = roster_id_team1.split(':')
            roster_ids_team2 = roster_id_team2.split(':')

            # Calculate average strike rate and 50+ scores for team1
            total_SR_team1 = 0
            count_team1 = 0
            total_fifty_plus_team1 = 0
            for player_id in roster_ids_team1:
                player_id = float(player_id)
                avg_SR, fifty_plus_count = calculate_avg_SR_and_50plus_count(player_id, match_dt, db_bat)
                total_SR_team1 += avg_SR
                total_fifty_plus_team1 += fifty_plus_count
                count_team1 += 1
            Avg_SR_team1.append(total_SR_team1 / count_team1 if count_team1 != 0 else 0)
            Fifty_plus_count_team1.append(total_fifty_plus_team1)

            # Calculate average strike rate and 50+ scores for team2
            total_SR_team2 = 0
            count_team2 = 0
            total_fifty_plus_team2 = 0
            for player_id in roster_ids_team2:
                player_id = float(player_id)
                avg_SR, fifty_plus_count = calculate_avg_SR_and_50plus_count(player_id, match_dt, db_bat)
                total_SR_team2 += avg_SR
                total_fifty_plus_team2 += fifty_plus_count
                count_team2 += 1
            Avg_SR_team2.append(total_SR_team2 / count_team2 if count_team2 != 0 else 0)
            Fifty_plus_count_team2.append(total_fifty_plus_team2)

            avg_SR = (Avg_SR_team1[-1] + Avg_SR_team2[-1]) / 2

            adv1 = 0
            adv2 = 0
            if avg_SR != 0:
                adv1 = Avg_SR_team1[-1] / avg_SR
                adv2 = Avg_SR_team2[-1] / avg_SR
                
            SR_Adv1.append(adv1)
            SR_Adv2.append(adv2)

        # Scale the strike rate advantage to a range of -1 to 1
        min_adv = min(SR_Adv1 + SR_Adv2)
        max_adv = max(SR_Adv1 + SR_Adv2)

        SR_Adv1_scaled = [(2 * (adv - min_adv) / (max_adv - min_adv)) - 1 if max_adv - min_adv != 0 else 0 for adv in SR_Adv1]
        SR_Adv2_scaled = [(2 * (adv - min_adv) / (max_adv - min_adv)) - 1 if max_adv - min_adv != 0 else 0 for adv in SR_Adv2]

        db_prm['Avg_SR_team1'] = Avg_SR_team1
        db_prm['Avg_SR_team2'] = Avg_SR_team2
        db_prm['Avg_SR_Diff'] = db_prm['Avg_SR_team1'] - db_prm['Avg_SR_team2']
        db_prm['SR_Adv_team1'] = SR_Adv1_scaled
        db_prm['SR_Adv_team2'] = SR_Adv2_scaled
        db_prm['Avg_SR_Adv_Diff'] = db_prm['SR_Adv_team1'] - db_prm['SR_Adv_team2']
       
        db_prm['Fifty_plus_count_team1'] = Fifty_plus_count_team1
        db_prm['Fifty_plus_count_team2'] = Fifty_plus_count_team2
        db_prm['Fifty_plus_diff']= db_prm['Fifty_plus_count_team1']- db_prm['Fifty_plus_count_team2']

        return db_prm
    
    db_prm = Add_StrikeRate(db_prm, db_mtch, db_bat)
    return db_prm

In [16]:
db_prm=SR1_Adder(db_prm, db_mtch, db_bat)

In [17]:
def Eco1_Adder(db_prm, db_mtch, db_bowl):
    
    def calculate_avg_economy_and_wickets(player_id, match_dt, db_bowl):
        # Get all matches for the player before the current match date
        player_matches = db_bowl[(db_bowl['bowler_id'] == player_id) & (db_bowl['match_dt'] < match_dt)]
        
        # Sort matches by date and get the last 5 matches
        player_matches_sorted = player_matches.sort_values(by='match_dt', ascending=False).head(5)
        
        # Calculate the average economy and count matches with 2 or more wickets
        if not player_matches_sorted.empty:
            avg_economy = player_matches_sorted['economy'].mean()
            wickets_2plus_count = (player_matches_sorted['wicket_count'] >= 2).sum()
            
            return avg_economy, wickets_2plus_count
        else:
            return 0, 0

    def Add_Economy(db_prm, db_mtch, db_bowl):
        Avg_Economy_team1 = []
        Avg_Economy_team2 = []
        Economy_Adv1 = []
        Economy_Adv2 = []
        Wickets_2plus_count_team1 = []
        Wickets_2plus_count_team2 = []

        for index, row in db_prm.iterrows():
            match_id = row['match id']
            match_dt = row['match_dt']
            team1_id = row['team1_id']
            team2_id = row['team2_id']
            roster_id_team1 = row['team1_roster_ids']
            roster_id_team2 = row['team2_roster_ids']

            # Split and store the player IDs from the roster
            roster_ids_team1 = roster_id_team1.split(':')
            roster_ids_team2 = roster_id_team2.split(':')

            # Calculate average economy and 2+ wickets counts for team1
            total_economy_team1 = 0
            count_team1 = 0
            total_wickets_2plus_team1 = 0
            for player_id in roster_ids_team1:
                player_id = float(player_id)
                avg_economy, wickets_2plus_count = calculate_avg_economy_and_wickets(player_id, match_dt, db_bowl)
                total_economy_team1 += avg_economy
                total_wickets_2plus_team1 += wickets_2plus_count
                count_team1 += 1
            Avg_Economy_team1.append(total_economy_team1 / count_team1 if count_team1 != 0 else 0)
            Wickets_2plus_count_team1.append(total_wickets_2plus_team1)

            # Calculate average economy and 2+ wickets counts for team2
            total_economy_team2 = 0
            count_team2 = 0
            total_wickets_2plus_team2 = 0
            for player_id in roster_ids_team2:
                player_id = float(player_id)
                avg_economy, wickets_2plus_count = calculate_avg_economy_and_wickets(player_id, match_dt, db_bowl)
                total_economy_team2 += avg_economy
                total_wickets_2plus_team2 += wickets_2plus_count
                count_team2 += 1
            Avg_Economy_team2.append(total_economy_team2 / count_team2 if count_team2 != 0 else 0)
            Wickets_2plus_count_team2.append(total_wickets_2plus_team2)

            avg_economy = (Avg_Economy_team1[-1] + Avg_Economy_team2[-1]) / 2

            adv1 = 0
            adv2 = 0
            if avg_economy != 0:
                adv1 = Avg_Economy_team1[-1] / avg_economy
                adv2 = Avg_Economy_team2[-1] / avg_economy
                
            Economy_Adv1.append(adv1)
            Economy_Adv2.append(adv2)

        # Scale the economy advantage to a range of -1 to 1
        min_adv = min(Economy_Adv1 + Economy_Adv2)
        max_adv = max(Economy_Adv1 + Economy_Adv2)

        Economy_Adv1_scaled = [(2 * (adv - min_adv) / (max_adv - min_adv)) - 1 if max_adv - min_adv != 0 else 0 for adv in Economy_Adv1]
        Economy_Adv2_scaled = [(2 * (adv - min_adv) / (max_adv - min_adv)) - 1 if max_adv - min_adv != 0 else 0 for adv in Economy_Adv2]

        db_prm['Avg_Economy_team1'] = Avg_Economy_team1
        db_prm['Avg_Economy_team2'] = Avg_Economy_team2
        db_prm['Avg_Economy_diff']= db_prm['Avg_Economy_team1']-db_prm['Avg_Economy_team2'] 
        db_prm['Economy_Adv_team1'] = Economy_Adv1_scaled
        db_prm['Economy_Adv_team2'] = Economy_Adv2_scaled
        db_prm['Avg_Economy_Adv1_Diff'] = db_prm['Economy_Adv_team1'] - db_prm['Economy_Adv_team2']
        
        # Scale Wickets_2plus_count to a range of 0 to 5
        db_prm['Wickets_2plus_count_team1'] = Wickets_2plus_count_team1
        db_prm['Wickets_2plus_count_team2'] = Wickets_2plus_count_team2
        db_prm['Wicket_2plus_diff']=db_prm['Wickets_2plus_count_team1'] -db_prm['Wickets_2plus_count_team2'] 

        return db_prm
    
    db_prm = Add_Economy(db_prm, db_mtch, db_bowl)
    return db_prm

In [18]:
db_prm=Eco1_Adder(db_prm, db_mtch, db_bowl)

In [19]:
def calculate_consistency_score(last_5_matches):
    consistency_score = 0
    match_reward = 0

    for match in reversed(last_5_matches):  # Start from the oldest to the newest
        if match == 'Won':
            match_reward = 2 if match_reward <= 0 else match_reward +1
        else:
            match_reward = -2 if match_reward >= 0 else match_reward - 1
        consistency_score += match_reward
    
    return consistency_score

def get_last_5_matches(team_id, match_dt, db_mtch):
    # Filter matches for the given team before the current match date
    team_matches = db_mtch[(db_mtch['match_dt'] < match_dt) & 
                           ((db_mtch['team1_id'] == team_id) | (db_mtch['team2_id'] == team_id))]
    
    # Sort matches by date and get the last 5 matches
    team_matches_sorted = team_matches.sort_values(by='match_dt', ascending=False).head(5)
    
    # Extract the results for the team in these matches
    match_results = []
    for _, match in team_matches_sorted.iterrows():
        if match['winner_id'] == team_id:
            result = 'Won'
        else:
            result = 'Lost'
        match_results.append(result)
    
    return match_results

def consistency_score_adder(db_prm, db_mtch):
    consistency_team1 = []
    consistency_team2 = []

    for index, row in db_prm.iterrows():
        match_dt = row['match_dt']
        team1_id = row['team1_id']
        team2_id = row['team2_id']

        # Get last 5 matches for team1 and team2
        match_results_team1 = get_last_5_matches(team1_id, match_dt, db_mtch)
        match_results_team2 = get_last_5_matches(team2_id, match_dt, db_mtch)

        # Calculate consistency scores
        consistency_team1.append(calculate_consistency_score(match_results_team1))
        consistency_team2.append(calculate_consistency_score(match_results_team2))

    # Scale the consistency scores to a range of -1 to 1
    min_score = min(consistency_team1 + consistency_team2)
    max_score = max(consistency_team1 + consistency_team2)

    consistency_team1_scaled = [(2 * (score - min_score) / (max_score - min_score)) - 1 if max_score - min_score != 0 else 0 for score in consistency_team1]
    consistency_team2_scaled = [(2 * (score - min_score) / (max_score - min_score)) - 1 if max_score - min_score != 0 else 0 for score in consistency_team2]

    db_prm['Consistency_Score_Team1'] = consistency_team1_scaled
    db_prm['Consistency_Score_Team2'] = consistency_team2_scaled
    db_prm['Consistency_Score_Diff'] = db_prm['Consistency_Score_Team1'] - db_prm['Consistency_Score_Team2']

    return db_prm

In [20]:
db_prm= consistency_score_adder(db_prm, db_mtch)

In [21]:
def find_openers(db_bat, match_id, roster_ids):
    openers = []
    
    if roster_ids is None or roster_ids == '':
        return openers
    
    roster_split = roster_ids.split(':')
    for player_id in roster_split:
        player_id = player_id.strip()  # Remove any leading/trailing whitespace
        if player_id:  # Check if player_id is not an empty string
            try:
                player_id = float(player_id)  # Convert player_id to float
                player_rows = db_bat[(db_bat['batsman_id'] == player_id) & (db_bat['match id'] == match_id)]
                if not player_rows.empty:
                    over_faced_first = player_rows['over_faced_first'].values[0]
                    if len(openers) < 2:
                        openers.append((player_id, over_faced_first))
                    else:
                        openers.sort(key=lambda x: x[1])
                        if over_faced_first < openers[-1][1]:
                            openers.pop()
                            openers.append((player_id, over_faced_first))
            except ValueError:
                continue  # Handle cases where player_id cannot be converted to float

    openers = [opener[0] for opener in openers]
    return openers


In [22]:
def Opener_SR_Adder(db_prm, db_mtch, db_bat):
    
    def calculate_SR(match_id, db_bat, openers):
        Avg_SR = 0
        count = 0
        for player_id in openers:
            matching_rows = db_bat[(db_bat['batsman_id'] == player_id) & (db_bat['match id'] == match_id)]
            if not matching_rows.empty:
                Avg_SR += matching_rows['strike_rate'].mean()
                count += 1

        if count == 0:
            return 0  # Return 0 if no valid openers found
             
        Avg_SR = Avg_SR / count
        return Avg_SR
    
    def Add_StrikeRate(db_prm, db_mtch, db_bat):
        Avg_SR_team1 = []
        Avg_SR_team2 = []
        
        for index, row in db_prm.iterrows():
            match_id = row['match id']
            match_dt = row['match_dt']
            team1_id = row['team1_id']
            team2_id = row['team2_id']
            roster_id_team1 = row['team1_roster_ids']
            roster_id_team2 = row['team2_roster_ids']

            df_team1, df_team2 = dataframe(db_mtch, team1_id, team2_id, match_dt)

            # Extract match and roster IDs for team1
            roster_id_last3rd_team1 = df_team1.loc['Roster_ID', 'Last3rd_Match']
            roster_id_last2nd_team1 = df_team1.loc['Roster_ID', 'Last2nd_Match']
            roster_id_last_team1 = df_team1.loc['Roster_ID', 'Last_Match']

            match_id_last3rd_team1 = df_team1.loc['Match_ID', 'Last3rd_Match']
            match_id_last2nd_team1 = df_team1.loc['Match_ID', 'Last2nd_Match']
            match_id_last_team1 = df_team1.loc['Match_ID', 'Last_Match']

            # Determine the openers for team1
            openers_last3rd_team1 = find_openers(db_bat, match_id_last3rd_team1, roster_id_last3rd_team1)
            openers_last2nd_team1 = find_openers(db_bat, match_id_last2nd_team1, roster_id_last2nd_team1)
            openers_last_team1 = find_openers(db_bat, match_id_last_team1, roster_id_last_team1)

            # Calculating the strike rate for openers for team1
            SRO_Last3rd_team1 = calculate_SR(match_id_last3rd_team1, db_bat, openers_last3rd_team1)
            SRO_Last2nd_team1 = calculate_SR(match_id_last2nd_team1, db_bat, openers_last2nd_team1)
            SRO_Last_team1 = calculate_SR(match_id_last_team1, db_bat, openers_last_team1)   
            Avg_SR1 = (SRO_Last3rd_team1 + SRO_Last2nd_team1 + SRO_Last_team1) / 3

            # Extract match and roster IDs for team2
            roster_id_last3rd_team2 = df_team2.loc['Roster_ID', 'Last3rd_Match']
            roster_id_last2nd_team2 = df_team2.loc['Roster_ID', 'Last2nd_Match']
            roster_id_last_team2 = df_team2.loc['Roster_ID', 'Last_Match']

            match_id_last3rd_team2 = df_team2.loc['Match_ID', 'Last3rd_Match']
            match_id_last2nd_team2 = df_team2.loc['Match_ID', 'Last2nd_Match']
            match_id_last_team2 = df_team2.loc['Match_ID', 'Last_Match']

            # Determine the openers for team2
            openers_last3rd_team2 = find_openers(db_bat, match_id_last3rd_team2, roster_id_last3rd_team2)
            openers_last2nd_team2 = find_openers(db_bat, match_id_last2nd_team2, roster_id_last2nd_team2)
            openers_last_team2 = find_openers(db_bat, match_id_last_team2, roster_id_last_team2)

            # Calculating the strike rate for openers for team2
            SRO_Last3rd_team2 = calculate_SR(match_id_last3rd_team2, db_bat, openers_last3rd_team2)
            SRO_Last2nd_team2 = calculate_SR(match_id_last2nd_team2, db_bat, openers_last2nd_team2)
            SRO_Last_team2 = calculate_SR(match_id_last_team2, db_bat, openers_last_team2)
            Avg_SR2 = (SRO_Last3rd_team2 + SRO_Last2nd_team2 + SRO_Last_team2) / 3
            
            Avg_SR_team1.append(Avg_SR1)
            Avg_SR_team2.append(Avg_SR2)

        db_prm['Opener_SR_team1'] = Avg_SR_team1
        db_prm['Opener_SR_team2'] = Avg_SR_team2
        db_prm['Opener_SR_Diff'] = db_prm['Opener_SR_team1'] - db_prm['Opener_SR_team2']

        return db_prm
    
    

    db_prm = Add_StrikeRate(db_prm, db_mtch, db_bat)
    return db_prm


In [23]:
db_prm=Opener_SR_Adder(db_prm, db_mtch, db_bat)

In [24]:
for index, row in db_prm.iterrows():
    if row['avg_runs_team1_last_5']==0 or row['avg_runs_team2_last_5']==0:
        row['avg_runs_team1_last_5']=0
        row['avg_runs_team2_last_5']=0
        row['run_avg_diff']=0
        row['team1_nrr']=0
        row['team2_nrr']=0
        row['nrr_diff']=0
        row['team1_wicket_runs']=0
        row['team2_wicket_runs']=0
        row['wicket_runs_diff']=0
        row['team1_wicket_runs']=0
        row['team2_wicket_runs']=0
        row['wicket_runs_diff']=0
        row['team1_wicket_overs']=0
        row['team2_wicket_overs']=0
        row['wicket_overs_diff']=0
        row['PPW_Avg_team1']=0
        row['PPW_Avg_team1']=0
        row['PPW_Avg_Diff']=0
        row['BoundaryFraction_team1']=0
        row['BoundaryFraction_team2']=0
        row['BoundaryFraction_Diff']=0

        others=['Avg_SR_team1','Avg_SR_team2','Avg_SR_Diff','SR_Adv_team1','SR_Adv_team2','Avg_SR_Adv_Diff','Fifty_plus_count_team1','Fifty_plus_count_team2',
                'Fifty_plus_diff','Avg_Economy_team1','Avg_Economy_team2','Avg_Economy_diff','Economy_Adv_team1','Economy_Adv_team2','Avg_Economy_Adv1_Diff',
                'Wickets_2plus_count_team1','Wickets_2plus_count_team2','Wicket_2plus_diff','Consistency_Score_Team1','Consistency_Score_Team2','Consistency_Score_Diff'
                ,'Opener_SR_team1','Opener_SR_team2','Opener_SR_Diff','Avg_Finishing_SR_team1','Avg_Finishing_SR_team2','Avg_Finishing_SR_Diff']
        for feature in others:
            row['feature']=0

In [29]:
others = [
    'Avg_SR_team1', 'Avg_SR_team2', 'Avg_SR_Diff', 'SR_Adv_team1', 'SR_Adv_team2', 'Avg_SR_Adv_Diff', 
    'Fifty_plus_count_team1', 'Fifty_plus_count_team2', 'Fifty_plus_diff', 'Avg_Economy_team1', 
    'Avg_Economy_team2', 'Avg_Economy_diff', 'Economy_Adv_team1', 'Economy_Adv_team2', 'Avg_Economy_Adv1_Diff', 
    'Wickets_2plus_count_team1', 'Wickets_2plus_count_team2', 'Wicket_2plus_diff', 'Consistency_Score_Team1', 
    'Consistency_Score_Team2', 'Consistency_Score_Diff', 'Opener_SR_team1', 'Opener_SR_team2', 
    'Opener_SR_Diff', 'Avg_Finishing_SR_team1', 'Avg_Finishing_SR_team2', 'Avg_Finishing_SR_Diff'
]

for index, row in db_prm.iterrows():
    if row['avg_runs_team1_last_5'] == 0 or row['avg_runs_team2_last_5'] == 0:
        db_prm.loc[index, 'avg_runs_team1_last_5'] = 0
        db_prm.loc[index, 'avg_runs_team2_last_5'] = 0
        db_prm.loc[index, 'run_avg_diff'] = 0
        db_prm.loc[index, 'team1_nrr'] = 0
        db_prm.loc[index, 'team2_nrr'] = 0
        db_prm.loc[index, 'nrr_diff'] = 0
        db_prm.loc[index, 'team1_wicket_runs'] = 0
        db_prm.loc[index, 'team2_wicket_runs'] = 0
        db_prm.loc[index, 'wicket_runs_diff'] = 0
        db_prm.loc[index, 'team1_wicket_overs'] = 0
        db_prm.loc[index, 'team2_wicket_overs'] = 0
        db_prm.loc[index, 'wicket_overs_diff'] = 0
        db_prm.loc[index, 'PPW_Avg_team1'] = 0
        db_prm.loc[index, 'PPW_Avg_team2'] = 0
        db_prm.loc[index, 'PPW_Avg_Diff'] = 0
        db_prm.loc[index, 'BoundaryFraction_team1'] = 0
        db_prm.loc[index, 'BoundaryFraction_team2'] = 0
        db_prm.loc[index, 'BoundaryFraction_Diff'] = 0
        
        for feature in others:
            db_prm.loc[index, feature] = 0


In [30]:
db_prm.to_csv("Data/FeatureEngNew.csv", index=False)

## Test Submission

In [26]:
db_NOtest= pd.read_csv('noData/Test Data.csv')
db_test['match_dt'] = pd.to_datetime(db_test['match_dt'])
db_test['ground_avg_runs_last15'].fillna(db_test['ground_avg_runs_last15'].median(), inplace=True)

FileNotFoundError: [Errno 2] No such file or directory: 'noData/Test Data.csv'

In [ ]:
add_nrr_features(db_test, team1_nrr, team2_nrr)
db_test = calculate_toss_advantage(db_mtch, db_test)
add_ratio_features(db_test, team1_stats, team2_stats)
add_batting_to_ground_avg_run_ratio(db_test)
db_test = PPW_adder(db_test,db_mtch,db_bat)
db_test=BoundaryRunsFraction_Adder(db_test, db_mtch, db_bat)
db_test=SR1_Adder(db_test, db_mtch,db_bat)
db_test=Eco1_Adder(db_test, db_mtch,db_bowl)
db_test= consistency_score_adder(db_test,db_mtch)
db_test=Eco1_Adder(db_test, db_mtch,db_bowl)

In [ ]:
db_test=Opener_SR_Adder(db_test, db_mtch,db_bat)

In [ ]:
# Ensure 'db_test' only contains the features we want to use for prediction
X_test_data = db_test[features]


# Handle any missing values in 'X_test_data' as appropriate (e.g., fill with mean or median)
X_test_data = X_test_data.fillna(X_test_data.mean())

# Ensure the data types are correct (int, float, bool, or category)
X_test_data = X_test_data.astype(float)

# Predict the probabilities on the test set
y_test_pred_proba = grid_search_gbm.predict_proba(X_test_data)[:, 1]

# Add the prediction score to db_test
db_test['win_pred_score'] = y_test_pred_proba

# Predict the classes on the test set
y_test_pred_class = grid_search_gbm.predict(X_test_data)

# Add the predicted winner's team ID to db_test
db_test['winner_id_pred'] = db_test.apply(lambda row: row['team1_id'] if row['win_pred_score'] > 0.5 else row['team2_id'], axis=1)

In [ ]:
X_train_data= db_prm[features]
y_train_pred_proba= grid_search_gbm.predict_proba(X_train_data)[:,1]
db_prm['win_pred_score']=y_train_pred_proba
y_train_pred_class = grid_search_gbm.predict(X_train_data)
db_prm['winner_id_pred'] = db_prm.apply(lambda row: row['team1_id'] if row['win_pred_score'] > 0.5 else row['team2_id'], axis=1)

In [ ]:
subfile1 = pd.read_csv('Data/sub_file1.csv')

In [ ]:
# Initialize new columns in subfile1
subfile1['win_pred_team_id'] = None
subfile1['win_pred_score'] = None

# Function to update predictions in subfile1 based on dataset type
def update_predictions(row, db_test, db_prm):
    match_id = row['match id']
    dataset_type = row['dataset_type']
    
    if dataset_type == 'r1':
        # Look up in db_test
        match_row = db_test[db_test['match id'] == match_id]
        if not match_row.empty:
            row['win_pred_team_id'] = match_row.iloc[0]['winner_id_pred']
            row['win_pred_score'] = match_row.iloc[0]['win_pred_score']
            
    elif dataset_type == 'train':
        # Look up in db_prm
        match_row = db_prm[db_prm['match id'] == match_id]
        if not match_row.empty:
            row['win_pred_team_id'] = match_row.iloc[0]['winner_id_pred']
            row['win_pred_score'] = match_row.iloc[0]['win_pred_score']
    
    return row

# Apply update_predictions function row-wise to update win_pred_team_id and win_pred_score
subfile1 = subfile1.apply(lambda row: update_predictions(row, db_test, db_prm), axis=1)

In [ ]:
# Feature importance
feature_importance = best_model.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importance})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

print("\nFeature Importance:")
print(feature_importance_df)

In [ ]:
subfile1.to_csv('subfile1_updated.csv', index=False)